In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData
from ipynb.fs.full.LabelTesting import makeOutThreeArr
import pandas as pd
from featurewiz import featurewiz

Imported 0.2.03 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



In [2]:
def compute_diffEnt(data):
    return scipy.stats.differential_entropy(data, axis=-1)

In [3]:
def get_features():
    matfiles = glob.glob('Preprocessed_EEG/*.mat')
    matfiles.sort()
    labels=loadmat(matfiles[15])
    label_list=np.ndarray.tolist(labels['label'])[0]
    x=getRawArrayData()
    selected_funcs = ['hjorth_complexity','hjorth_mobility',('diffEnt', compute_diffEnt),'std','skewness','mean','variance','ptp_amp','kurtosis','rms','hurst_exp','katz_fd']
    allData=np.zeros((1,13))
    for clips in x[0]:
        cur_clip=mne.make_fixed_length_epochs(x[0][clips], duration=1, preload=True,verbose=0)
        epoch_array=mne.Epochs.get_data(cur_clip)
        cur_label=label_list[clips]
        cur_func=[selected_funcs[0]]
        extracted_data=extract_features(epoch_array,200,cur_func)
        one_feature=np.reshape(extracted_data[0],(62,1))
        for row in range (1,extracted_data.shape[0]):
            cur_time_read = np.reshape(extracted_data[row],(62,1))
            one_feature = np.vstack((one_feature,cur_time_read))
        com=np.zeros((one_feature.shape[0],1))
        com=np.hstack((com,one_feature))
        for i in range(1,len(selected_funcs)):
            cur_func=[selected_funcs[i]]
            extracted_data=extract_features(epoch_array,200,cur_func)
            one_feature=np.reshape(extracted_data[0],(62,1))
            for j in range (1,extracted_data.shape[0]):
                cur_time_read = np.reshape(extracted_data[j],(62,1))
                one_feature = np.vstack((one_feature,cur_time_read))
            com = np.hstack((com,one_feature))
        label_col=np.zeros((com.shape[0],1))+cur_label
        com=np.delete(com, 0, axis=1)
        com = np.hstack((com,label_col))
        allData=np.vstack((allData,com))
    col_name= ['hjorth_complexity','hjorth_mobility','diffEnt','std','skewness','mean','variance','ptp_amp','kurtosis','rms','hurst_exp','katz_fd','label']
    df = pd.DataFrame(allData, columns =col_name)
    features = featurewiz(df, target='label', corr_limit=0.70, verbose=0)
    return features[0]